In [105]:
# import libraries
import numpy as np
import glob, os
import torch
os.chdir('../..')

# Main

In [103]:
def main():
    truth_values, file_names = get_file_information()
    training_matrix = process_data(file_names)
    training_matrix = numpy_to_torch(training_matrix)
    

In [104]:
main()

(590, 2100)


In [106]:
def numpy_to_torch(array):
    return torch.from_numpy(array)

# Get all file names

In [96]:
def get_file_information():
    truth_values = []
    file_names = []
    base_directory = os.getcwd() + '/train'
    subdirectories = ['anger', 'despair', 'happiness', 'neutral', 'sadness']
    for directory in subdirectories:
        os.chdir(base_directory + '/' + directory)
        for file_name in glob.glob("*.csv"):
            truth_values.append(directory)
            file_names.append(os.getcwd() + '/' + file_name)
    return truth_values, file_names

# Load data into numpy

In [70]:
# Load CSV and return np.ndarray
def csv_to_numpy(filename):
    raw_data = open(filename, 'rt')
    data = np.loadtxt(raw_data, delimiter=",",skiprows=1)
    return data


# Delete rows beyond 140 or add extra rows if not 140 length

In [71]:
# Need standard length for file for matrix
# Returns flattened array for big matrix
def format_np_and_flatten(data):
    num_rows = data.shape[0]
    if num_rows > 140: # delete rows
        data = np.delete(data, slice(140,num_rows), axis=0)
    else: # add rows of zeros
        num_rows_to_add = 140 - num_rows
        rows = np.zeros((num_rows_to_add, data.shape[1]))
        data = np.append(data, rows, axis=0)
    return data.flatten()

# Process data

In [87]:
def process_data(file_names):
    training_matrix = []
    for filename in file_names:
        data = csv_to_numpy(filename)
        data = format_np_and_flatten(data)
        training_matrix.append(data)
    return np.array(training_matrix)

In [83]:
training_matrix

[array([  0.      ,   0.      , -13.66599 , ...,   6.692688,   3.497931,
         10.05002 ])]

In [84]:
training_matrix = np.array(training_matrix)

In [85]:
training_matrix

array([[  0.      ,   0.      , -13.66599 , ...,   6.692688,   3.497931,
         10.05002 ]])